In [1]:
import os
import numpy as np
import time
import pickle

In [2]:
def smooth_landmark(curr_landmark, prev_landmark, next_landmark):
    landmarks = [curr_landmark, prev_landmark, next_landmark]
    s_landmarks = []
    
    for l in landmarks:
        if l is not None:
            s_landmarks.append(l)
            
    if len(s_landmarks) == 0:
        return None
    elif len(s_landmarks) == 1:
        return s_landmarks[0]
    elif len(s_landmarks) == 2:
        return (s_landmarks[0] + s_landmarks[1]) / 2
    elif len(s_landmarks) == 3:
        return (s_landmarks[0] + s_landmarks[1] + s_landmarks[2]) / 3

In [3]:
def get_face_moves(mid_landmarks):
    face_moves = []
#     the first move is zero
    face_moves.append(np.array([0,0]))
    for i in range(len(mid_landmarks)-1):
        move = mid_landmarks[i+1] - mid_landmarks[i]
        face_moves.append(move)
    return face_moves

In [4]:
def get_mid_landmark(landmarks):
    mid_landmarks = []
    for i in range(len(landmarks)):
        curr_landmark = prev_landmark = next_landmark = None
        if i == 0:            
            if landmarks[i] is not None:
                curr_landmark = landmarks[i]
            if landmarks[i+1] is not None:
                next_landmark = landmarks[i+1]
        elif i == (len(landmarks)-1):            
            if landmarks[i] is not None:
                curr_landmark = landmarks[i]
            if landmarks[i-1] is not None:
                prev_landmark = landmarks[i-1] 
        else:
            if landmarks[i] is not None:
                curr_landmark = landmarks[i]
            if landmarks[i-1] is not None:
                prev_landmark = landmarks[i-1] 
            if landmarks[i+1] is not None:
                next_landmark = landmarks[i+1]
                
        landmark_s = smooth_landmark(curr_landmark, prev_landmark, next_landmark)
        mid_landmark = [0, 0]
        if landmark_s is not None:
            mid_landmark[0] = landmark_s[0,:].mean()
            mid_landmark[1] = landmark_s[1,:].mean()
        mid_landmarks.append(np.array(mid_landmark))
    return mid_landmarks

In [7]:
annotation_path = '/nas/database/Big5/gt/annotation_test.pkl'
input_dir = '/nas/project_data/B1_Behavior/hakim/FI/3DDFA/test'
output_dir = '/nas/project_data/B1_Behavior/hakim/FI/pose_face_moves/test'

with open(annotation_path, 'rb') as f:
    annotation = pickle.load(f, encoding='latin1')
videos = list(annotation['extraversion'].keys())

for i, video in enumerate(videos):
    video_name = video.rsplit(".")[0] + "." + video.rsplit(".")[1]
    input_name = video_name + '.pickle'
    output_name = video_name + '.npy'
    input_path = os.path.join(input_dir, input_name)
    output_path = os.path.join(output_dir, output_name)
    print(f'Processing {i} {video_name}')
    
    with open(input_path, 'rb') as f:
        tddfa_output = pickle.load(f, encoding='latin1')
    
    landmarks = []
    for item in tddfa_output:
        if item is None:
            landmarks.append(None)
        else:
            landmarks.append(item['landmark'])
    
    mid_landmarks = get_mid_landmark(landmarks)
    face_moves = get_face_moves(mid_landmarks)
    features = []
    
    for i in range(len(tddfa_output)):
        if tddfa_output[i] is None:
            feature = [0, 0, 0] + list(face_moves[i])
        else:
            feature = tddfa_output[i]['pose'] + list(face_moves[i])
        features.append(feature)   
        
    features = np.array(features)
    
    with open(output_path, 'wb') as f:
        np.save(f, features)

Processing 0 htH89DBizno.004
Processing 1 p_wf-KszNlk.001
Processing 2 MuYYY3XaJ7Q.001
Processing 3 0MB91ku0eEw.005
Processing 4 WpEZOSrENL0.003
Processing 5 C2Y9Puk3Obk.004
Processing 6 ask-ZFRztf8.003
Processing 7 TSGpD2NBeCQ.005
Processing 8 54JawR1x0II.004
Processing 9 9n8dNi-ERQ0.001
Processing 10 lNG_XdqMf0g.004
Processing 11 Jy716zClCKs.004
Processing 12 w_ExyXsnw2A.005
Processing 13 wKBObhJXCSQ.000
Processing 14 p6UMx8dwHZQ.005
Processing 15 R42dmIPCXwo.000
Processing 16 3tPq9fNOXZQ.000
Processing 17 -6otZ7M-Mro.002
Processing 18 9UEBU_yiTPs.000
Processing 19 ouYYWCY__5A.004
Processing 20 69BopbFc34U.002
Processing 21 SIpQm4G4DDg.003
Processing 22 u_qvbRpm88k.000
Processing 23 KnbEca4Ibwk.004
Processing 24 Hm5K_K0v718.002
Processing 25 p_kPnCD7WzU.003
Processing 26 9cj8rrj1wSk.003
Processing 27 x6Tluj4dJTU.003
Processing 28 bnM9Ow52x3o.003
Processing 29 L4uFD6434Pc.000
Processing 30 j1idNSoB7tA.005
Processing 31 mFz5Fbs97QA.004
Processing 32 0G2o3fik36I.000
Processing 33 OZlpEN

Processing 273 0iQIfGnKflc.003
Processing 274 ZGox7tevC6A.000
Processing 275 Uu-NbXUPr-A.003
Processing 276 7knYNR1tAig.001
Processing 277 4kYaYIcuLpM.000
Processing 278 ZIyfXdCxN9E.001
Processing 279 d1eh3dHNi1Y.005
Processing 280 2TXrDZgbDHE.004
Processing 281 pOtWKOF07iU.005
Processing 282 ZO5JoyVTat8.003
Processing 283 YJ5CINQ8v_E.002
Processing 284 IL5VgGakSNw.004
Processing 285 6-_1-vBNl_0.000
Processing 286 zF3Uk2OAJAE.003
Processing 287 isaP2l3rlBQ.004
Processing 288 nQtiqrceBhg.004
Processing 289 pKm9lxa4cmg.000
Processing 290 syJo_sm8IqQ.001
Processing 291 4XdZDodpzac.004
Processing 292 shQmRW4CrGE.003
Processing 293 Nv0-x4K9YFI.003
Processing 294 Pr-RpZ4gYCM.004
Processing 295 qAisk1lgv2I.000
Processing 296 9-lptN92A68.003
Processing 297 2AV8m02PjdU.005
Processing 298 J8YpOUmN_aI.002
Processing 299 Z_STuUHKQTg.001
Processing 300 5IqtX-uq28E.004
Processing 301 Pb7aNc6Kd40.000
Processing 302 W48yW9XTI4c.000
Processing 303 cgGCyBMdGrA.005
Processing 304 2d6btbaNdfo.000
Processi

Processing 541 oontew1LiUU.000
Processing 542 wK_ExIjn5Q8.000
Processing 543 _7hGh6VU4IU.001
Processing 544 isqmQc9AfBc.002
Processing 545 Jcw9NZkMn6I.000
Processing 546 4nJLbAKgwfw.003
Processing 547 vYkHHVeNK_M.000
Processing 548 oYiWW-Qc0rs.004
Processing 549 4CSV8L7aVik.003
Processing 550 heE4ZZXwehI.005
Processing 551 7ZnijRlK5-E.005
Processing 552 sf29orxf4X0.005
Processing 553 9RfE2-aTvaM.004
Processing 554 w4cZStXMUII.004
Processing 555 53QFyec0uN0.001
Processing 556 FGkS4vJYOpM.001
Processing 557 ZeGT9pSG734.000
Processing 558 wYixHjntI-4.000
Processing 559 gkhbUDIK-YM.002
Processing 560 rCxlfusNJlY.003
Processing 561 LKc69Et2Jg8.003
Processing 562 3S72dDIm1fM.005
Processing 563 RWfY3Xk0XPQ.001
Processing 564 ySEeioc339E.001
Processing 565 aQVGVhcMBtE.002
Processing 566 U2RemWUyjts.004
Processing 567 P7poMmhYnM0.002
Processing 568 kL-CeaXG9jM.005
Processing 569 PVm7Om88BSw.002
Processing 570 dl9IT3OvI0o.000
Processing 571 p-V78q8dtYo.000
Processing 572 pnY7YOgdl54.001
Processi

Processing 806 vqLtOsoNPnU.005
Processing 807 KsToJAKi7xQ.001
Processing 808 4a6-tNo-NDM.001
Processing 809 4lIbWq27O84.003
Processing 810 Sdqg-jn_tBk.002
Processing 811 r2HcJYjGK5s.000
Processing 812 W0Ay5uul8j0.000
Processing 813 cJs_LosR-u8.002
Processing 814 r9TaaVTo8Y8.005
Processing 815 DctPau1p7JM.000
Processing 816 mfGhtr0zy3k.003
Processing 817 phjVMKGYBKs.002
Processing 818 fiEEjrKhlNU.000
Processing 819 knpgk7LAMb0.004
Processing 820 3gmc2kLV4Bo.003
Processing 821 7kO7jYYUVWg.001
Processing 822 8hGOOynn3ZU.000
Processing 823 scifP1EXSdo.000
Processing 824 De4i7-FX9Og.005
Processing 825 D4hPHPp1W-Q.003
Processing 826 Ay7bZ7Sgx9Q.002
Processing 827 2uAJJH7B5aw.002
Processing 828 QBZieJiOl1w.000
Processing 829 7qGYGbIg45c.003
Processing 830 Qz_cjgCtDcM.002
Processing 831 XnSbc0tEzQ0.005
Processing 832 UvX6ctu2ObM.000
Processing 833 AJIphVRpYrg.001
Processing 834 fyXiORXmWHs.005
Processing 835 UMMhvyjgRnM.002
Processing 836 HoJXPFp9hYo.002
Processing 837 Vyktsh14Kww.002
Processi

Processing 1072 B8kzwm6ZaUo.003
Processing 1073 FcR9VhCxBb0.005
Processing 1074 -DOqN0d8KHw.005
Processing 1075 rIHcq2E67Nc.001
Processing 1076 n6JNJurop2U.001
Processing 1077 xcMwMi4fdis.000
Processing 1078 M5_x5J-H2I0.002
Processing 1079 RF3JnDmv8WY.005
Processing 1080 CGfr9jl8w4Y.002
Processing 1081 13KtmgntgQw.004
Processing 1082 jlDYn537Pd0.005
Processing 1083 4vdJGgZpj4k.000
Processing 1084 BStHNKV820Q.005
Processing 1085 BLc_GvsbI1U.001
Processing 1086 mK8ebTJyrCI.004
Processing 1087 qVrXROy4Ais.005
Processing 1088 w0CQ55sKmck.000
Processing 1089 ihmTGSMvQM4.003
Processing 1090 54JawR1x0II.000
Processing 1091 s6N1pv5lzrU.001
Processing 1092 AjCI_w9Szks.004
Processing 1093 gj3-xvhPeWg.004
Processing 1094 NoHext7JXYU.005
Processing 1095 phjVMKGYBKs.004
Processing 1096 eXtskPoi58A.004
Processing 1097 U2RemWUyjts.005
Processing 1098 oe1PZxExhzc.002
Processing 1099 7Y4cUPFEt-4.003
Processing 1100 CFk4v2Kd5Ns.000
Processing 1101 NXG4n1pr4Ok.005
Processing 1102 dLltwqkvsNg.000
Processi

Processing 1330 h9jIdLBaMEo.003
Processing 1331 cWvZnl5XjwI.002
Processing 1332 nOCSTnpB_F4.004
Processing 1333 n4NkVZ1_dfk.001
Processing 1334 upgv7xmOb1I.000
Processing 1335 xq0-QVHxp-Q.005
Processing 1336 40rn4A64Bdk.004
Processing 1337 QpA0gj1y6KE.002
Processing 1338 qBpv0BfiGQk.000
Processing 1339 FlAGm93fJa0.003
Processing 1340 be0DQawtVkE.002
Processing 1341 WpEZOSrENL0.000
Processing 1342 36sZ__mv_KY.000
Processing 1343 _kK9tGN883Y.005
Processing 1344 cgp1OzTOq1o.001
Processing 1345 S-jhg6Y5OrE.000
Processing 1346 Ie2y_yOeKpo.002
Processing 1347 xqGyF2T3pso.004
Processing 1348 n877YOU6Qy0.000
Processing 1349 IzsP0TRzFbo.001
Processing 1350 lShOveDL2dw.003
Processing 1351 h6LOjpCRXtY.002
Processing 1352 cnsvc0alWOA.005
Processing 1353 Vk3scj5vhd4.003
Processing 1354 Y1LXjxAQihQ.000
Processing 1355 rW3hFpfGMHs.002
Processing 1356 SxKLweLx_ZE.001
Processing 1357 2SzC9dm4Yy4.000
Processing 1358 0axZSeaUbfs.000
Processing 1359 Ld8ouPJpdDU.000
Processing 1360 BBSB9OJdb0Q.000
Processi

Processing 1587 OdwK_oMdfSU.004
Processing 1588 QTtTAFqT-no.005
Processing 1589 4vJ69g7gAH4.002
Processing 1590 gOS-CneQsxI.001
Processing 1591 YBsKLg3GMrE.001
Processing 1592 CESTtNvDpww.002
Processing 1593 fipvPdFbKN4.003
Processing 1594 cx8IycWgMVs.001
Processing 1595 BmDClgdWlH4.001
Processing 1596 bAxlTgobVHE.005
Processing 1597 J6RHl6U6Jcw.003
Processing 1598 MDoZUDTSayc.004
Processing 1599 Lz3hYPF6aIM.001
Processing 1600 QS7TwjzkYiU.001
Processing 1601 nYCuMoC5ibQ.003
Processing 1602 y2UF49ddapI.000
Processing 1603 ly_0m16694Q.002
Processing 1604 bEXDNfLBy1I.000
Processing 1605 19pTUX8KfYM.003
Processing 1606 PsqpkFKX8co.003
Processing 1607 KNx7_ajRwDI.005
Processing 1608 fG-hvGTHoec.003
Processing 1609 naAGy30bnwA.004
Processing 1610 4JjpfL4y3XM.002
Processing 1611 uMCRfie1lkc.000
Processing 1612 IQNNEfYYYiw.000
Processing 1613 ev42vngXERU.003
Processing 1614 XJj34u5IzU0.005
Processing 1615 836kYlEs9GI.005
Processing 1616 gWCgqVn3m3k.003
Processing 1617 n5Jeeqfj2_U.001
Processi

Processing 1849 ilOi-sQo9HI.002
Processing 1850 nGe8dIc-ecc.001
Processing 1851 ogSLrHVeqLI.002
Processing 1852 AWe6ZZUmlZw.002
Processing 1853 fEix0DPOWtg.004
Processing 1854 oC-qxnrmJws.004
Processing 1855 fKrX-KXgXYM.005
Processing 1856 DcVwxxbKGF4.004
Processing 1857 VnUkBYQkJms.000
Processing 1858 A1lbD_OCuj4.004
Processing 1859 m2uxFfpb5xU.003
Processing 1860 50gokPvvMs8.004
Processing 1861 uStuc1OX0f0.005
Processing 1862 a38rz5AIslE.005
Processing 1863 2SzC9dm4Yy4.005
Processing 1864 wJl7DbsbrDA.002
Processing 1865 lErjGxBb1Bk.001
Processing 1866 UEYKBC_SuzU.003
Processing 1867 8hGOOynn3ZU.002
Processing 1868 R6NTuhZtOks.004
Processing 1869 Prb5j6jLKBs.005
Processing 1870 8qDKGnABATw.005
Processing 1871 KYkfjIZEScM.002
Processing 1872 KTdbKiGFzJY.001
Processing 1873 R9JcmvLBbTs.001
Processing 1874 QS7TwjzkYiU.005
Processing 1875 KYvMsMLNGvY.001
Processing 1876 fBMKlYUjA9E.002
Processing 1877 ASDpn9yOYAo.001
Processing 1878 69EsTnCVCp0.002
Processing 1879 qLbOPbx33dk.000
Processi